In [1]:
import requests
import re
import sys
import json
from bs4 import BeautifulSoup
from tqdm import tqdm

import pandas as pd
import datetime
from datetime import datetime

df1 = pd.read_csv(r"D:\git\team5a\NEWS_TEAM_5\Data\Naver_social_URL_split_11\Naver_social_URL_1.csv")

data = pd.DataFrame()

for url_addr in df1['URL']:
    oid = url_addr.split('/')[5]
    aid = url_addr.split('/')[6].split('?')[0]

    r = requests.get("https://cbox5.apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=view_society&pool=cbox5&_cv=20221122141410&_callback=jQuery33109815095164386811_1669873780035&lang=ko&country=KR&objectId=news"+str(oid)+","+str(aid)+"&pageSize=100&indexSize=10&listType=OBJECT&pageType=more&page=1&initialize=true&useAltSort=true&replyPageSize=20&sort=favorite&includeAllStatus=true",


    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',

        'referer': url_addr
    })

    soup = BeautifulSoup(r.content, 'html.parser')

    json_text = re.findall('\{.*?\)+;', soup.get_text())[0][:-2]

    info = json.loads(json_text)['result']['commentList']

    for comment in info:
        UserID = comment['userIdNo']
        UserName = comment['maskedUserName']
        WritedAt = comment['modTime']
        WritedAt = WritedAt[0:10]+' '+WritedAt[11:16]
        Content = comment['contents'].replace("\n", ' ').strip()
        
        comments_info_df = pd.DataFrame({'URL' : url_addr,
                                    'UserID' : UserID,
                                    'UserName' : UserName, 
                                    'WritedAt' : WritedAt,
                                    'Content' : Content},index = [0])
        data = pd.concat([data, comments_info_df])


In [ ]:
data[:5]


In [4]:
data.to_csv(r"./Naver_social_comments_1", index=False, escapechar='\\')